## 1. Imports


In [330]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

In [331]:
data_path = '../results/synthetic_balTestSet/bb/'
folders= ['dt','gnb','lgr','gbt']
#folders= ['dt','gnb']

## 2. CSVs

In [332]:
def impact_csvs(data_path= 'data/results/',b_or_w = 'Black', folders= ['dt','gnb','lgr','gbt']):

    col_names_eg = []
    col_names_gs = []

    for i,f in enumerate(folders):
        if b_or_w == 'Black':
            path = f'{data_path}{f}/{f}_black_results.csv'
        else:
            path = f'{data_path}{f}/{f}_white_results.csv'

        df = pd.read_csv(path,index_col=0)
        df = df.reset_index()

        col_names_eg.append(f'EG+{f.upper()}')
        col_names_gs.append(f'GS+{f.upper()}')

        if i == 0:
            joined_df = df.iloc[:,-1]
        else:
            joined_df = pd.concat([joined_df, df.iloc[:,-1]], axis=1)

    joined_df.set_axis(folders, axis=1)


    # split dataframe after the two reduction algorithms
    df_eg = joined_df.iloc[:6,:]
    df_gs = pd.concat([joined_df.iloc[0:1,:],joined_df.iloc[6:,:]])

    # set new index
    df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_eg.set_index('Constraint',inplace=True)
    df_gs['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_gs.set_index('Constraint',inplace=True)
    df_eg.columns = col_names_eg
    df_gs.columns = col_names_gs

    df_final = pd.concat([df_eg, df_gs], axis=1)
    print('Group: ',b_or_w,'\n DataFrame: \n',df_final)
    print('A')
    df_final.to_csv(f'{data_path}/{b_or_w}_DI.csv')
    print('B')


In [333]:
impact_csvs(data_path,'Black', folders= ['dt','gnb','lgr','gbt'])
impact_csvs(data_path,'White', folders= ['dt','gnb','lgr','gbt'])

Group:  Black 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated  17.28   18.74   17.05   17.38  17.28   18.74   17.05   17.38
DP            3.23    0.60    2.12    3.59  12.09   14.36    5.65   12.09
EO           14.84   12.50   13.43   14.86  17.28   18.74   17.05   17.38
EOO          13.77   11.78   11.92   13.91  12.62   12.87    7.56   12.09
FPER         15.62   14.08   14.29   15.62  12.09   16.95   17.05   12.09
ERP          17.28   18.29   17.09   17.38  17.28   18.84   17.38   17.38
A
B
Group:  White 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated   9.58    2.08    8.45    9.11   9.58    2.08    8.45    9.11
DP           17.68   16.52   17.57   18.05  16.05    9.94   15.82   16.55
EO           13.95   12.13   12.49   13.63   9.58 

C:\Users\HaRi\AppData\Local\Temp\ipykernel_48052\620886726.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\AppData\Local\Temp\ipykernel_48052\620886726.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


## 3. FP/TP/TN/FN Ratios

In [334]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

Classifier:  dt 
 DataFrame: 
 ID        egdpB  egdpW  egeoB  egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                 
FN        0.038  0.072  0.069  0.056   0.086   0.045    0.071    0.038   
FP        0.202  0.075  0.110  0.109   0.085   0.165    0.103    0.118   
TN        0.298  0.425  0.390  0.391   0.415   0.335    0.397    0.382   
TP        0.462  0.428  0.431  0.444   0.414   0.455    0.429    0.462   

ID        egtprpB  egtprpW  gsdpB  gsdpW  gseoB  gseoW  gserpB  gserpW  \
Category                                                                 
FN          0.065    0.051  0.059  0.055  0.086  0.022   0.086   0.022   
FP          0.118    0.099  0.133  0.095  0.085  0.154   0.085   0.154   
TN          0.382    0.401  0.367  0.405  0.415  0.346   0.415   0.346   
TP          0.435    0.449  0.441  0.445  0.414  0.478   0.414   0.478   

ID        gsfprpB  gsfprpW  gstprpB  gstprpW  testB  testW  unmitB  unmitW  
Ca

In [335]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

Classifier:  dt 
 DataFrame: 
 ID         egdpB   egdpW   egeoB   egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                     
FN         283.0   537.0   512.0   420.0   639.0   334.0    531.0    287.0   
FP        1513.0   563.0   820.0   816.0   635.0  1236.0    772.0    882.0   
TN        2223.0  3173.0  2916.0  2920.0  3101.0  2500.0   2964.0   2854.0   
TP        3453.0  3199.0  3224.0  3316.0  3097.0  3402.0   3205.0   3449.0   

ID        egtprpB  egtprpW   gsdpB   gsdpW   gseoB   gseoW  gserpB  gserpW  \
Category                                                                     
FN          489.0    380.0   440.0   409.0   639.0   164.0   639.0   164.0   
FP          885.0    741.0   993.0   708.0   635.0  1153.0   635.0  1153.0   
TN         2851.0   2995.0  2743.0  3028.0  3101.0  2583.0  3101.0  2583.0   
TP         3247.0   3356.0  3296.0  3327.0  3097.0  3572.0  3097.0  3572.0   

ID        gsfprpB  gsfprpW  gst

# Analyzing Scores

#### Extractig Scores from csv into dataframes

In [336]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}
dfs_eg = {}
dfs_gs = {}
for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    df_eg = pd.concat([df.iloc[:,:4],df.filter(like='eg')],axis=1)
    df_gs = pd.concat([df.iloc[:,:4],df.filter(like='gs')],axis=1)
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white
    dfs_eg[f] = df_eg
    dfs_gs[f] = df_gs
print(classifier_dfs['dt'])

      testB  testW  unmitB  unmitW  egdpB  egdpW  egeoB  egeoW  egtprpB  \
0       750    837     825     850    825    850    825    837      825   
1       647    782     722     850    722    850    722    782      722   
2       508    605     508     455    508    605    508    455      508   
3       444    348     444     348    444    348    444    348      444   
4       767    701     842     776    842    776    842    776      842   
...     ...    ...     ...     ...    ...    ...    ...    ...      ...   
7467    658    562     733     412    733    562    733    412      733   
7468    771    379     621     379    621    379    621    379      621   
7469    565    505     640     505    640    505    640    505      640   
7470    632    379     707     379    707    379    707    379      707   
7471    475    527     475     527    475    527    475    527      475   

      egtprpW  egfprpB  egfprpW  egerpB  egerpW  gsdpB  gsdpW  gseoB  gseoW  \
0         850      8

### Checking if normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [337]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

Classifier: dt
Check for norm Distributions done
Classifier: gnb
Check for norm Distributions done
Classifier: lgr
Check for norm Distributions done
Classifier: gbt
Check for norm Distributions done


### Mann Whitney U test:

“a two-sample rank test for the difference between two population medians . . . It assumes that the data are independent random samples from two populations that have the same shape.”

In [338]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

#### Variance of Distributions EG vs. GS
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [339]:
p_vals = pd.DataFrame(data={'Constraints': []})
p_signi = pd.DataFrame(data={'Constraints': []})
    
for (c1,df_eg),(c2,df_gs) in zip(dfs_eg.items(),dfs_gs.items()):
    col_signi = []
    col_vals = []
    idx = []
    
    for col_eg,col_gs in zip(df_eg,df_gs):
        
        idx.append(col_eg[2:])
        
        data_eg=df_eg[col_eg].dropna(axis=0)
        data_gs=df_gs[col_gs].dropna(axis=0)
        
        _,p = stats.mannwhitneyu(data_eg, data_gs)
        
        col_vals.append(p)
        if p< 0.05:
            col_signi.append('s')
        else:
            col_signi.append(' ')
            
    p_signi[c1] = col_signi
    p_vals[c1] = col_vals
    
p_vals['Constraints'] = idx
p_vals = p_vals.set_index('Constraints')
p_signi['Constraints'] = idx
p_signi = p_signi.set_index('Constraints')

p_vals = p_vals.round(decimals=3)
print(p_signi)
    
p_vals.to_csv(f'{mwu_path}p_eg_gs.csv')
p_signi.to_csv(f'{mwu_path}significanz_eg_gs.csv')


            dt gnb lgr gbt
Constraints               
stB                       
stW                       
mitB                      
mitW                      
dpB          s   s   s   s
dpW              s        
eoB          s   s   s   s
eoW          s   s   s   s
tprpB                s    
tprpW                     
fprpB        s   s   s   s
fprpW                s    
erpB                      
erpW             s   s    


#### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [340]:
def p_values_mwu_1model(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [341]:
for col in classifier_dfs['dt']:
    print('\nC:',col)
    p_values_mwu_1model(classifier_dfs,col)


C: testB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: testW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: unmitB
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: unmitW
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: egdpB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                     s
lgr                      
gbt             s        

C: egdpW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C

#### Variance of Distributions unmitigated v Mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [342]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [343]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB gnbB lgrB gbtB
Constraints                   
test          s    s    s    s
unmit                         
egdp          s    s    s    s
egeo          s    s    s    s
egtprp        s    s    s    s
egfprp             s    s     
egerp                         
gsdp          s    s    s    s
gseo                          
gstprp        s    s    s    s
gsfprp        s    s         s
gserp                         

White:
            dtW gnbW lgrW gbtW
Constraints                   
test          s         s    s
unmit                         
egdp          s    s    s    s
egeo          s    s    s    s
egtprp        s    s    s    s
egfprp        s    s    s    s
egerp                         
gsdp          s    s    s    s
gseo                          
gstprp        s    s    s    s
gsfprp        s    s         s
gserp              s    s     


Fairness Metrics 

In [344]:
# Absolute numbers
dfs = {} # list for pandas dfs
datasets = ['00', '0b', '0i', 'b0','bb','bi','i0','ib','ii']
datasets = ['00', '0b']
folders = ['dt']
#pd.set_option('display.max_columns', None)
for _,f in enumerate(folders):
    for _,ds in enumerate(datasets):
        path = f'../results/synthetic_balTestSet/{ds}/{f}/{f}_overall_results.csv'
        df = pd.read_csv(path)
        df = pd.concat([df.iloc[:,:2], df.iloc[:,13:]],axis= 1)
        df = df.set_index('Run')
        df2 = df.iloc[:,:1]
        df2['Fairness Metric Value Difference to Unmitigated'] = df.iloc[:,1]
        df2.iloc[0,1] = None

        for i in range(1,6):
            df2.iloc[i,1] = df.iloc[i,i]-df.iloc[0,i]
            df2.iloc[i+5,1] = df.iloc[i+5,i]-df.iloc[0,i]
        print(df2)  
        dfs[f] = df2
        #print('\nClassifier: ',f,'\n DataFrame: \n',df)
        #df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

                        Acc  Fairness Metric Value Difference to Unmitigated
Run                                                                         
dt Unmitigated        81.61                                              NaN
dt EG DP Mitigated    75.02                                             0.98
dt EG EO Mitigated    79.45                                           -17.09
dt EG TPRP Mitigated  79.78                                           -21.72
dt EG FPRP Mitigated  82.71                                           -19.61
dt EG ERP Mitigated   80.13                                            -0.29
dt GS DP Mitigated    67.66                                            25.15
dt GS EO Mitigated    81.61                                             0.00
dt GS TPRP Mitigated  77.26                                           -17.49
dt GS FPRP Mitigated  81.61                                             0.00
dt GS ERP Mitigated   81.61                                             0.00

C:\Users\HaRi\AppData\Local\Temp\ipykernel_48052\963955507.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Fairness Metric Value Difference to Unmitigated'] = df.iloc[:,1]
C:\Users\HaRi\AppData\Local\Temp\ipykernel_48052\963955507.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Fairness Metric Value Difference to Unmitigated'] = df.iloc[:,1]


In [345]:
#stats.pearsonr(data_unmiti, data_miti)